In [1]:
!pip install meteocalc

  Created wheel for meteocalc: filename=meteocalc-1.1.0-cp37-none-any.whl size=8195 sha256=7fad87e5a23e6f757ada71308d1715d337507a8477aac076b596442693622036
  Stored in directory: /root/.cache/pip/wheels/9e/34/13/83d36ecc28837e3c2a5b696542e697538e7c1025382f4ded55
Successfully built meteocalc


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/ashrae.zip ashrae.zip
!unzip -q ashrae.zip


Mounted at /content/drive


importations des librairies nécessaires

In [18]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.figure_factory as ff

%matplotlib inline

plt.style.use("ggplot")
from IPython.core.pylabtools import figsize
from plotly.offline import init_notebook_mode,iplot,plot
%matplotlib inline
init_notebook_mode(connected=True)
plt.style.use("ggplot")

from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import train_test_split

%run utils.py
%run feature_enginering.py

In [19]:
building = pd.read_csv('/content/building_metadata.csv')
weather_train = pd.read_csv('/content/weather_train.csv')
train = pd.read_csv('/content/train.csv')

# convertir du kwh vers kbtu
train.loc[(train.meter == 0),'meter_reading'] = train['meter_reading'] * 0.2931

In [20]:
def join_data(data , weather_data , building_data):
    building_data = reduce_mem_usage(building_data)
    weather_data  = reduce_mem_usage(weather_data)
    data = reduce_mem_usage(data)

    data = (data.merge(building_data, on='building_id', how='left')) #on joint les données ci-dessus a partir des clés
    data = data.merge(weather_data, how='left', on = ['site_id', 'timestamp'], validate='many_to_one')

    return data

In [21]:
print(train.shape)

(20216100, 4)


In [22]:
train = join_data(train , weather_train , building)
train['log_meter_reading'] = np.log(train['meter_reading'] + 1)
train = train.drop(['meter_reading'] , axis = 1)
del weather_train
gc.collect()

8

In [23]:
for col in  train.columns:
    print(col , ' - ',train[col].isnull().values.sum() )

building_id  -  0
meter  -  0
timestamp  -  0
site_id  -  0
primary_use  -  0
square_feet  -  0
year_built  -  12127645
floor_count  -  16709167
air_temperature  -  96658
cloud_coverage  -  8825365
dew_temperature  -  100140
precip_depth_1_hr  -  3749023
sea_level_pressure  -  1231669
wind_direction  -  1449048
wind_speed  -  143676
log_meter_reading  -  0


In [24]:
cols_to_drop = ['year_built' , 'floor_count' ,
                'cloud_coverage' , 'precip_depth_1_hr' , 
                'wind_direction' , 'sea_level_pressure']

cols_to_scale = ['square_feet' , 'air_temperature' 
                 , 'dew_temperature' , 'wind_speed']

cols_to_encode = ['primary_use']


cols_to_fill_mean = ['air_temperature', 'dew_temperature'  , 'wind_speed' ]

Filler = Data_Filler(cols_to_fill_mean,[])

Filler.fit(train)
train = Filler.transform(train)

ScalerEncoder = Data_Scaler_Encoder(cols_to_scale ,
                                    cols_to_drop , 
                                    cols_to_encode)
ScalerEncoder.fit(train)
train = ScalerEncoder.transform(train)

In [25]:
for col in  train.columns:
    print(col , ' - ',train[col].isnull().values.sum() )

building_id  -  0
meter  -  0
timestamp  -  0
site_id  -  0
primary_use  -  0
square_feet  -  0
air_temperature  -  0
dew_temperature  -  0
wind_speed  -  0
log_meter_reading  -  0


In [26]:
def time_features(df) :
  df["timestamp"] = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
  
  df["hour"] = df["timestamp"].dt.hour
  df["dayofweek"] = df["timestamp"].dt.weekday
  df['month'] = df['timestamp'].dt.month 

  df =  df.drop(['timestamp'] , axis= 1)
  return df

train = time_features(train)

In [27]:
train.head()

,building_id,meter,site_id,primary_use,square_feet,air_temperature,dew_temperature,wind_speed,log_meter_reading,hour,dayofweek,month
0,0,0,0,0,-0.856658,0.857421,1.225247,-1.554017,0.0,0,4,1
1,1,0,0,0,-0.896882,0.857421,1.225247,-1.554017,0.0,0,4,1
2,2,0,0,0,-0.874209,0.857421,1.225247,-1.554017,0.0,0,4,1
3,3,0,0,0,-0.717912,0.857421,1.225247,-1.554017,0.0,0,4,1
4,4,0,0,0,0.075327,0.857421,1.225247,-1.554017,0.0,0,4,1


In [28]:
train = reduce_mem_usage(train)

y = train['log_meter_reading'].values
train = train.drop(['log_meter_reading'],axis=1).values

Random Forest Regressor

In [29]:
from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor(n_estimators=25,min_samples_split=200,min_samples_leaf=100)
RF_model = RF_model.fit(train, y)

In [30]:
del train,y
gc.collect()

978

### Prediction

In [31]:
weather_test = pd.read_csv('/content/weather_test.csv')
x_test = pd.read_csv('/content/test.csv')
building = pd.read_csv('/content/building_metadata.csv')

x_test = join_data(x_test , weather_test , building)

del weather_test , building
x_test = Filler.transform(x_test)
x_test = ScalerEncoder.transform(x_test)
x_test = time_features(x_test)

In [32]:
for col in  x_test.columns:
    print(col , ' - ',x_test[col].isnull().values.sum() )

row_id  -  0
building_id  -  0
meter  -  0
site_id  -  0
primary_use  -  0
square_feet  -  0
air_temperature  -  0
dew_temperature  -  0
wind_speed  -  0
hour  -  0
dayofweek  -  0
month  -  0


In [33]:
row_id = x_test.row_id.values
x_test = x_test.drop(['row_id'] , axis= 1)
x_test = reduce_mem_usage(x_test)

In [34]:
y_pred = np.empty(len(x_test))
y_pred[0:int(len(x_test)/3)] = np.exp(RF_model.predict(x_test.iloc[0:int(len(x_test)/3)]))-1
y_pred[int(len(x_test)/3):2*int(len(x_test)/3)] = np.exp(RF_model.predict(x_test.iloc[int(len(x_test)/3):2*int(len(x_test)/3)]))-1
y_pred[2*int(len(x_test)/3):] = np.exp(RF_model.predict(x_test.iloc[2*int(len(x_test)/3):]))-1
y_pred[x_test.meter==0] = y_pred[x_test.meter==0] / 0.2931

In [35]:
pred_final = pd.DataFrame({'row_id': row_id, 'meter_reading': y_pred})

In [36]:
pred_final

,row_id,meter_reading
0,0,0.005498
1,1,0.168104
2,2,0.009201
3,3,0.000000
4,4,0.016952
...,...,...
41697595,41697595,5.017687
41697596,41697596,4.811669
41697597,41697597,8.532532
41697598,41697598,180.276597


In [37]:
compression_opts = dict(method='zip',archive_name='out2.csv')  

pred_final.to_csv('out2.zip', index=False,compression=compression_opts)

In [38]:
!cp out2.zip /content/drive/MyDrive/out2.zip